# Sound Object Playback

The purpose of this script is to break out some 5-second sound objects from the Metatone touch archive and play them back on a device.

In [1]:
#%matplotlib inline
from __future__ import print_function
import os
import time
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import seaborn as sns
import random
from touch_performance_player import TouchPerformancePlayer
import OSC
import sys
sys.path.append("./MetatoneClassifier/classifier/")
import metatone_classifier
import generate_posthoc_gesture_score
import pickle

def clean_sound_object(frame):
    """Cleans up sound object frames by removing unneeded 
        columns and changing times to differences."""
    first_time = frame.index[0].to_pydatetime()
    output = frame[['x_pos','y_pos','velocity']]
    output['time'] = output.index
    output.time = (output.time - first_time).apply(timedelta.total_seconds)
    output.time = output.time.fillna(0)
    output = output.rename(columns={'x_pos': 'x', 'y_pos': 'y', 'velocity': 'velocity', 'time': 'time'})
    return output

def five_second_frame(row, frame):
    """
    Returns a dataframe of five seconds length before "time".
    """
    end_time = row.name
    frame = frame.between_time((end_time + timedelta(seconds=-5)).time(), end_time.time())
    if frame.empty:
        return np.nan
    return frame

def add_gestures_to_objects_series(objects_series):
    """Classifies gestures from a series of sound-object frames."""
    feature_vectors = objects_series.apply(generate_posthoc_gesture_score.feature_vector_from_frame).apply(pd.Series)
    out = pd.DataFrame({'objects':objects_series})
    feature_vectors = feature_vectors[metatone_classifier.FEATURE_VECTOR_COLUMNS]
    preds = generate_posthoc_gesture_score.CLASSIFIER.classifier.predict(feature_vectors)
    out['gestures'] = preds
    return out

def convert_log_frame_to_classified_sound_objects(log):
    """Converts a touch-log into a dataframe of sound-objects and classified gestures."""
    sound_object_frames = []
    for n in log.device_id.unique():
        log_frame = log[log.device_id == n]
        times = pd.DataFrame(log_frame.index,index=log_frame.index).resample('1s').count()
        sound_objects = times.apply(five_second_frame, axis=1, frame=log_frame).dropna()
        sound_objects = add_gestures_to_objects_series(sound_objects)
        print("Classified",len(sound_objects.index),"sound objects.")
        sound_object_frames.append(sound_objects)
    out = pd.concat(sound_object_frames)
    print("Converting touch logs to sound_object format")
    out.objects = out.objects.apply(clean_sound_object)
    print("Total sound objects classified is:",len(out.index))
    return out

### Loading Gesture Classifier.           ###
### Classifier file successfully loaded.  ###


### Load up data

- Loads metatone logs.
- Checks if output directory exists, otherwise creates it.

In [2]:
log_files = []
performances = []

output_directory = "individual-perfs-metatone-format/"
output_fileending = "-indperf.csv"

if not os.path.exists(output_directory):
    print("Creating Individual Perf Directory")
    os.makedirs(output_directory)

for local_file in os.listdir("data"):
    if local_file.endswith("-touches.csv"):
        log_files.append("data/" + local_file)
# for log in log_files:
    # do some stuff
    
log_frames = []
print("Loading all the frames.")
for log in log_files:
    log_frames.append(pd.DataFrame.from_csv(log,parse_dates=True,header=0))
print("Done Loading", len(log_frames), "logs.")

Loading all the frames.
Done Loading 163 logs.


## Loading gesture to sound-object data

In [3]:
# Load or generate the sound_object to classified object frame and save as a pickle.
gesture_to_object_filename = "./gesture_to_sound_object_dataframe.pickle"
try:
    pickle_file = open(gesture_to_object_filename, "rb")
    cla_frame = pickle.load(pickle_file)
    pickle_file.close()
    print("### Classified sound objects successfully loaded.  ###")
except:
    print("### Error loading sound objects, classifying again.")
    cla_frame = convert_log_frame_to_classified_sound_objects(log_frames[120])
    with open(gesture_to_object_filename, 'wb') as f:
        pickle.dump(cla_frame, f, pickle.HIGHEST_PROTOCOL)
    print("### Classified sound objects successfully generated and saved.  ###")

### Error loading sound objects, classifying again.
Classified 521 sound objects.
Classified 523 sound objects.
Classified 515 sound objects.
Classified 468 sound objects.
Converting touch logs to sound_object format
Total sound objects classified is: 2027
### Classified sound objects successfully generated and saved.  ###


## Test out the sound_object playback

### Setup and start playing

In [4]:
## When we get home, make this work.
## Need to find out address of iPads manually using bonjour browser.

player = TouchPerformancePlayer('charles','192.168.0.36',51200)
player.setPerformances(cla_frame)
player.startPlaying()

('Initialising a performance player for', 'charles', 'at', '192.168.0.36:51200')
Setting up new sound_object dataframe:
2    591
4    385
3    381
7    223
8    209
1    188
6     29
5     21
Name: gestures, dtype: int64
('Available Gestures:', [8, 7, 4, 3, 1, 2, 5, 6, 0])
('Starting Playback with gesture:', 0)
('Playing perf-object with gesture:', 0, 'on:', 'charles')
('Playing perf-object with gesture:', 0, 'on:', 'charles')
('Playing perf-object with gesture:', 0, 'on:', 'charles')


### Update Gesture to send

In [10]:
player.updateGesture(4)

('Playing perf-object with gesture:', 4, 'on:', 'charles')


In [18]:
p = {'a':1}

x = ['a','b','c','d']
enumerate(x)

### Stop playing

In [11]:
player.stopPlaying()

('Stopping playback and cancelling timers for:', 'charles')
